### K means

a) (1-dimensional clustering) Walk through Lloyd's algorithm step by step on the following dataset:

`[0, .5, 1.5, 2, 6, 6.5, 7]` (note: each of these are 1-dimensional data points)

Given the initial centroids:

`[0, 2]`

a. randomly pick k centers
b. assign each point in the dataset to closest center
c. compute new centers as means of each cluster
d. repeat b and c until convergence

a0. 0 and 2 are picked
b0. {0, 0.5}, {1.5, 2, 6, 6.5, 7}
c0. means are 0.25 and 4.6

loops:
{0, 0.5, 1.5, 2}, {6, 6.5, 7} means are 2 and 6.5
The cluster means stagnate here, hence algo ends.

b) Describe in plain english what the cost function for k means is.

It is the minimized sum of distances in that k points cluster.

c) For the same number of clusters K, why could there be very different solutions to the K means algorithm on a given dataset?

With K means algo there can be multiple solutions as inital centers can change the local optimal.

d) Does Lloyd's Algorithm always converge? Why / why not?

Proof by contridiction:
Case 1: the entire process continues forever
The set is finite, there are only so many partitions avaliable, hence it will end because there is a minimum.
Case 2: the clustering process loops
Not possible, because we are refining the cost at each loop since the algo actively decreases or at least stagnates the cost at each loop. If it stagnates that means the algo has the best clustering based on cost.

e) Follow along in class the implementation of Kmeans

In [ ]:
import numpy as np
from PIL import Image as im
import matplotlib.pyplot as plt
import sklearn.datasets as datasets

centers = [[0, 0], [2, 2], [-3, 2], [2, -4]]
X, _ = datasets.make_blobs(n_samples=300, centers=centers, cluster_std=1, random_state=0)

class KMeans():

    def __init__(self, data, k):
        self.data = data
        self.k = k
        self.assignment = [-1 for _ in range(len(data))]
        self.snaps = []

    def distance(self, x, y):
        return np.linalg.norm(x - y)    
    
    def snap(self, centers):
        TEMPFILE = "temp.png"

        fig, ax = plt.subplots()
        ax.scatter(X[:, 0], X[:, 1], c=self.assignment)
        ax.scatter(centers[:,0], centers[:, 1], c='r')
        fig.savefig(TEMPFILE)
        plt.close()
        self.snaps.append(im.fromarray(np.asarray(im.open(TEMPFILE))))

    def initialize(self):
        return self.data[np.random.choice(range(len(self.data)), self.k, replace=False)]
    
    def assign(self, centers):
        for i in range(len(self.data)):
            mini = self.distance(centers[0], self.data[i])
            self.assignment = 0
            for j in range(len(centers)):
                dist = self.distance(centers[j], self.data[i])
                if dist < mini: 
                    mini = dist
                    self.assignment[i] = j
        return 
    
    def is_diff_clusters(self, centers, new_centers):
        for i in range(len(centers)):
            if self.distance(centers[i], new_centers[i]):
                return True
        return False
    
    def get_centers(self):
        for i in set(self.assignment):
            cluster = []
            for j in range(len(self.data)):
                if self.assignment[j] == i:
                    cluster.append(self.data[j])
            centers.append(np.mean(cluster))
        return centers

    def lloyds(self):
        centers = self.initialize()
        self.assign(centers)
        self.snap(centers)
        new_centers = self.get_centers()
        while self.is_diff_clusters(centers, new_centers):
            self.assign(new_centers)
            centers = new_centers
            self.snap(centers)
            new_centers = self.get_centers()
        return 
            

kmeans = KMeans(X, 6)
kmeans.lloyds()
images = kmeans.snaps

images[0].save(
    'kmeans.gif',
    optimize=False,
    save_all=True,
    append_images=images[1:],
    loop=0,
    duration=500
)